In [1]:
%load_ext autoreload
%autoreload 2

# Data Processing

In [2]:
import pandas as pd
import json
from datasets import load_dataset

In [3]:
act_labels = ["dummy", "inform", "question", "directive", "commissive"]
emotion_labels = ["no_emotion", "anger", "disgust", "fear", "happiness", "sadness", "surprise"]

In [4]:
dataset = load_dataset("daily_dialog")

In [5]:
for split, split_dataset in dataset.items():
    split_dataset.to_csv(f"daily_dialog-{split}.csv")

Creating CSV from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
df1 = pd.read_csv("daily_dialog-train.csv")
df2 = pd.read_csv("daily_dialog-validation.csv")
df3 = pd.read_csv("daily_dialog-test.csv")

df = pd.concat([df1, df2, df3])

df1 = df2 = df3 = None

In [70]:
import spacy
from spacy_cleaner import processing, Cleaner

model = spacy.load("en_core_web_sm")
cleaner = Cleaner( 
    model,
    #processing.remove_stopword_token,
    #processing.replace_punctuation_token,
    processing.mutate_lemma_token,
)

In [71]:
def parse_dialog(dialog, remove_punctuation=True):
    turns = dialog.replace("\'", "").replace("[", "").replace("]", "").replace('"', "").split("\n")
    return turns

def parse_label_numbers(label_numbers):
    label_numbers = label_numbers.replace("[", "").replace("]", "").split(" ")
    return list(map(int, label_numbers))

In [72]:
rows = []

for i, row in df.iterrows():
    turns = parse_dialog(row["dialog"])
    act = parse_label_numbers(row["act"])
    emotion = parse_label_numbers(row["emotion"])
    for sequence, turn in enumerate(turns):
        rows.append({
            "original_index": i,
            "sequence": sequence,
            "act_label": act_labels[act[sequence]],
            "emotion_label": emotion_labels[emotion[sequence]],
            "text": turn
        })

turns_df = pd.DataFrame.from_records(rows)

In [80]:
text = turns_df.iloc[190]["text"]
print(text)
cleaner.clean([text])

  Havent you heard that he is in prison ?   What ? Beg your pardon . 


Cleaning Progress: 100%|██████████| 1/1 [00:00<00:00, 197.70it/s]


['have not you hear that he be in prison ? what ? beg your pardon .']

In [81]:
cleaned_texts = cleaner.clean(turns_df["text"])

Cleaning Progress: 100%|██████████| 90010/90010 [01:32<00:00, 972.47it/s] 


In [82]:
turns_df["cleaned_text"] = cleaned_texts

In [83]:
turns_df

,original_index,sequence,act_label,emotion_label,text,cleaned_text
0,0,0,directive,no_emotion,"Say , Jim , how about going for a few beers af...","say , Jim , how about go for a few beer after ..."
1,0,1,commissive,no_emotion,You know that is tempting but is really not ...,you know that be tempting but be really not go...
2,0,2,question,no_emotion,What do you mean ? It will help us to relax .,what do you mean ? it will help we to relax .
3,0,3,question,no_emotion,Do you really think so ? I dont . It will ju...,do you really think so ? I do not . it will ju...
4,0,4,question,no_emotion,I guess you are right.But what shall we do ?...,I guess you be right . but what shall we do ? ...
...,...,...,...,...,...,...
90005,999,7,question,surprise,are you kidding ? Can you afford it ? Do you...,be you kid ? can you afford it ? do you think ...
90006,999,8,directive,no_emotion,"never mind that , Ill take care of it . Are ...","never mind that , Ill take care of it . be you..."
90007,999,9,commissive,no_emotion,"yeah , I think so .","yeah , I think so ."
90008,999,10,inform,happiness,ok . Ill make the arrangements . It will be ...,ok . Ill make the arrangement . it will be gre...


 # Prepare for Classification


In [84]:
label_type = "act"

classification = turns_df[[f"{label_type}_label", "cleaned_text"]]

In [85]:
classification.rename({f"{label_type}_label": "label", "cleaned_text": "text"}, axis=1, inplace=True)

/tmp/ipykernel_1652978/2370742413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classification.rename({f"{label_type}_label": "label", "cleaned_text": "text"}, axis=1, inplace=True)


In [86]:
labels = classification["label"].sort_values().drop_duplicates().reset_index(drop=True)

In [87]:
label2id = labels_map = dict(list(zip(labels, labels.index)))
id2label = dict((v,k) for k,v in labels_map.items())

In [88]:
labels_map

{'commissive': 0, 'directive': 1, 'inform': 2, 'question': 3}

In [89]:
num_labels = len(labels)

In [90]:
from sklearn.model_selection import train_test_split

In [91]:
from datasets import Dataset, DatasetDict

In [92]:
train_df, val_df = train_test_split(classification, test_size=0.2)
train_df["label"].replace(labels_map, inplace=True)
val_df["label"].replace(labels_map, inplace=True)
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset = Dataset.from_pandas(val_df.reset_index(drop=True))

In [93]:
dataset = DatasetDict({"train": train_dataset, "test": val_dataset})

In [94]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=num_labels)

model.config.id2label = id2label
model.config.label2id = label2id

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [95]:
from transformers import TrainingArguments

output_dir = f"../../models/distilroberta-daily_dialog-{label_type}"

In [96]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

/opt/conda/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [97]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [98]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,
    eval_strategy="epoch",
    num_train_epochs=3
)

In [99]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

Map:   0%|          | 0/72008 [00:00<?, ? examples/s]

Map:   0%|          | 0/18002 [00:00<?, ? examples/s]

In [68]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [69]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.223400,1.236967,0.435229
2,1.166100,1.166132,0.487612


KeyboardInterrupt: 

In [ ]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
tokenizer.save_vocabulary(output_dir)

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model=output_dir, return_all_scores=True)

In [ ]:
classifier.model.config.id2label = id2label
classifier.model.config.label2id = labels_map

In [ ]:
classifier.model.config

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model=output_dir, return_all_scores=True)
classifier.model.config.id2label = id2label
classifier.model.config.label2id = label2id

def classify(text):
    results = classifier(text)
    max_score = max(results[0], key=lambda x:x["score"])
    return max_score["label"]

In [ ]:
classify("What's going on?")

In [ ]:
classify("I don't think so.")

In [ ]:
classify("You think so? I don't know, really.")

In [ ]:
classify("Do as I say.")

In [ ]:
classify("Tell that guy to shut up")

In [ ]:
classify("I'm sick of seeing you here")

In [ ]:
classify("Take the book and read it")

In [ ]:
classify("Do you think it's okay?")

In [ ]:
classify("It's tempting but it's not good for our fitness")

In [ ]:
classify("Sometimes I think life is not worth living.")

In [ ]:
classify("Find another person.")

In [ ]:
classify("Re-train on this dataset")

In [ ]:
classify("Do this")

In [ ]:
classify("Get this from google")

In [ ]:
classify('why not go again to celebrate out one-year anniversary ? We can go to the same beach , stay in the same hotel and enjoy a dinner in the same restaurant .')

In [ ]:
classify("Why bother")

In [ ]:
classify("Who's the author of this article")

In [ ]:
classify("Where was this")